Los datasets utilizados son del portal:

https://datos.gob.mx/busca/dataset/incidencia-delictiva-del-fuero-comun

Importamos las librerias que se requieren

In [1]:
import numpy as np 
import pandas as pd
import altair as alt

### Obtenemos los datos de archivos '.csv'  y los convertimos en dataframes.

Para el dataframe1 eliminamos la columna INEGI Y con una mascara obtenemos los delitos cometidos desde 1997 hasta 2014, esto debido a que el dataframe2 inicia en el año 2015

Tambien renombramos algunas columnas esto con el fin de que sean lo mas parecida posible al otro dataframe. Esto despues de explorar los datos y ver que columnas son mas parecidas.

Esto lo hice debido a que aunque ambos datasets son muy similares, difieren en la metodologia de clasificación con la cual registraron los archivos.

In [13]:
df1 = pd.read_csv('./data/IDEFC_1.csv',encoding='latin-1')
df1 = df1.drop(['INEGI'],axis=1)
df1 = df1[ df1.AÑO <= 2014]
df1 = df1.rename({  'ENTIDAD':'Entidad', 'MODALIDAD':'Tipo de delito', 'TIPO':'Modalidad', 'SUBTIPO':'Subtipo de delito',
                    'ENERO':'Enero', 'FEBRERO':'Febrero', 'MARZO':'Marzo', 'ABRIL':'Abril',
                    'MAYO':'Mayo', 'JUNIO':'Junio', 'JULIO':'Julio', 'AGOSTO':'Agosto',
                    'SEPTIEMBRE':'Septiembre', 'OCTUBRE':'Octubre','NOVIEMBRE':'Noviembre','DICIEMBRE':'Diciembre'
                }, axis=1)
#ver las ultimas 5 filas
df1.tail()

,AÑO,Entidad,Tipo de delito,Modalidad,Subtipo de delito,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
38011,2014,ZACATECAS,DELITOS SEXUALES (VIOLACION),VIOLACION,VIOLACION,6,7,10,20,10,15,10,17,13,11,9,13
38012,2014,ZACATECAS,OTROS DELITOS,AMENAZAS,AMENAZAS,14,24,26,27,28,49,41,32,20,35,21,22
38013,2014,ZACATECAS,OTROS DELITOS,ESTUPRO,ESTUPRO,1,6,2,2,0,2,1,0,3,1,5,2
38014,2014,ZACATECAS,OTROS DELITOS,OTROS SEXUALES,OTROS SEXUALES,1,1,2,1,1,2,1,2,2,5,3,1
38015,2014,ZACATECAS,OTROS DELITOS,RESTO DE LOS DELITOS (OTROS),RESTO DE LOS DELITOS (OTROS),195,182,208,215,249,309,219,229,239,287,224,245


En el dataframe2 eliminamos 2 columnas que no seran requeridas.

Con la funcion .tail() observamos que apartir de agosto/2021 tenemos datos NaN. Es decir los registros llegan hasta Julio del 2021


In [9]:
df2 = pd.read_csv('./data/IDEFC_NM.csv',encoding='latin-1')
df2 = df2.drop(['Bien jurídico afectado','Clave_Ent'],axis=1)
df2.tail()

,Año,Entidad,Tipo de delito,Subtipo de delito,Modalidad,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
21947,2021,Zacatecas,Falsificación,Falsificación,Falsificación,11,11,10,6,7,7,2,NaN,NaN,NaN,NaN,NaN
21948,2021,Zacatecas,Contra el medio ambiente,Contra el medio ambiente,Contra el medio ambiente,0,0,2,0,1,1,0,NaN,NaN,NaN,NaN,NaN
21949,2021,Zacatecas,Delitos cometidos por servidores públicos,Delitos cometidos por servidores públicos,Delitos cometidos por servidores públicos,24,21,26,32,31,37,33,NaN,NaN,NaN,NaN,NaN
21950,2021,Zacatecas,Electorales,Electorales,Electorales,12,4,6,14,32,55,12,NaN,NaN,NaN,NaN,NaN
21951,2021,Zacatecas,Otros delitos del Fuero Común,Otros delitos del Fuero Común,Otros delitos del Fuero Común,113,133,125,151,145,141,119,NaN,NaN,NaN,NaN,NaN


Ademas con la funcion .info() Observamos que las columnas desde Enero hasta Octubre son de tipo object apesar de tener numeros almacenados en  sus casillas. Podemos deducir que algunos de sus elementos contiene caracteres o ',' 

In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21952 entries, 0 to 21951
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Año                21952 non-null  int64  
 1   Entidad            21952 non-null  object 
 2   Tipo de delito     21952 non-null  object 
 3   Subtipo de delito  21952 non-null  object 
 4   Modalidad          21952 non-null  object 
 5   Enero              21952 non-null  object 
 6   Febrero            21952 non-null  float64
 7   Marzo              21952 non-null  float64
 8   Abril              21952 non-null  float64
 9   Mayo               21952 non-null  float64
 10  Junio              21952 non-null  float64
 11  Julio              21952 non-null  float64
 12  Agosto             21952 non-null  float64
 13  Septiembre         21952 non-null  float64
 14  Octubre            21952 non-null  float64
 15  Noviembre          21952 non-null  float64
 16  Diciembre          219

Tendremos que modificar algunas columnas para que sean consideradas durante los calculos.

Para ello obtenemos los nombres de las columnas de dichos meses y con un ciclo for convertiremos el tipo de columna object a float.

Esto provoca que los elementos no numericos sean sustituidos por el tipo NaN. A continuación sustituimos los elementos NaN por 0.

In [15]:
meses = df2.columns.values
meses = meses[6:16]
for i in meses: 
    df2[i] = pd.to_numeric(df2[i], errors='coerce')
df2 = df2.fillna(0)

Obtenemos las categorias de los delitos de cada dataframe Esto para elegir con que tipo de delito vamos a trabajar

In [17]:
type_delito1 = df1['Tipo de delito'].unique()
type_delito2 = df2['Tipo de delito'].unique()

print(f" El df1 tiene {type_delito1.size} tipos de delito")
print(f" El df2 tiene {type_delito2.size} tipos de delito")

 El df1 tiene 10 tipos de delito
 El df2 tiene 40 tipos de delito


Vemos los tipos de delitos que existen en el df1 con el cual trabajaremos a continuación.

Se trabajara con los casos de 'ROBO' como casos Base.

In [18]:
type_delito1

array(['DELITOS PATRIMONIALES', 'DELITOS SEXUALES (VIOLACION)',
       'HOMICIDIOS', 'LESIONES', 'OTROS DELITOS',
       'PRIV. DE LA LIBERTAD (SECUESTRO)', 'ROBO COMUN',
       'ROBO DE GANADO (ABIGEATO)', 'ROBO EN INSTITUCIONES BANCARIAS',
       'ROBO EN CARRETERAS'], dtype=object)

Creamos DataFrame que contenga unicamente tipos de delito con la cadena 'HOMICIDIOS'

In [35]:
df_typeDelito1 = df1.loc[df1['Tipo de delito'].str.contains('HOMICIDIOS')]
#df_typeDelito1

Con el df anterior obtenemos el total de delitos cometidos

Esto con ayuda de .groupby().sum()
Y posteriormente sumamos los meses para tener el total 

In [36]:
df_TD = df_typeDelito1.groupby('Tipo de delito').sum()
df_TD = df_TD.drop(['AÑO'],axis=1)
df_TD = df_TD.sum(axis=1)
df_TD

Tipo de delito
HOMICIDIOS    565283
dtype: int64

## 8 Entidades con mayor numero de delitos cometidos

Ahora queremos obtener el total de incidencias de cada Entidad del país y ordenadas de forma Descendente para visualizar las 8 con mayor cantidad de incidencias.

Esto lo obtenemos sencillamente con .groupby('Entidad') que agrupara la tabla a solo 32 filas que son las entidades del país.

Eliminamos

Sumamos los meses para tener el total por entidad.

Ordenamos ascendentemente

Invertimos para tenerlo descendente y finalmente filtramos los 8 primeros


In [38]:
df_entity = df_typeDelito1.groupby('Entidad').sum()
df_entity = df_entity.drop(['AÑO'],axis=1)
df_entity = df_entity.sum(axis=1)
df_entity = df_entity.sort_values()# Ordenar 
df_entity = df_entity[::-1]# Invertir
df_entity = df_entity[:8]# Filtrar

df_entity

Entidad
MEXICO              84323
JALISCO             36296
MICHOACAN           32281
GUERRERO            31717
CHIHUAHUA           31283
CHIAPAS             30383
CIUDAD DE MEXICO    27827
VERACRUZ            26764
dtype: int64

## La importancia de Groupby()

### El codigo siguiente repite el mismo proceso que realiza una sola linea de codigo

df_entity = df_typeDelito1.groupby('Entidad').sum()

In [66]:
""" 
#Comprobar si los valores con cuerdan con data set respecto a mes y año
dfxPais = dfrobo[ dfrobo.Clave_Ent == 31]
mes = dfxPais.DICIEMBRE
sumxMes = np.add.reduce(mes)
sumxMes
"""
#Obtenemos el total de entidades y sus respectivas claves.
entidades = df1['Entidad'].unique()
dftarget = pd.DataFrame(entidades,columns=['Entidad'])

#Array con columnas de meses
meses = df1.columns.values
meses = meses[5:]

matrizR= np.zeros(1)
for i in meses:
    totalrobo = np.zeros(1)
    for entidad in entidades:
        dfxPais = df_typeDelito1[ df_typeDelito1.Entidad == entidad]
        sumxMes = np.add.reduce(dfxPais[i])
        totalrobo = np.append(totalrobo,sumxMes)
    
    totalrobo = np.delete(totalrobo,0)
    matrizR = np.concatenate([matrizR, totalrobo])

matrizR = np.delete(matrizR,0)
matrizR = matrizR.reshape((12, 32)).transpose()
dfmeses = pd.DataFrame(matrizR,columns=meses)
dftarget = pd.concat([dftarget, dfmeses], axis=1)
dftarget

,Entidad,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,AGUASCALIENTES,331.0,365.0,370.0,304.0,368.0,357.0,285.0,288.0,284.0,320.0,298.0,415.0
1,BAJA CALIFORNIA,1046.0,876.0,894.0,916.0,943.0,923.0,950.0,956.0,1026.0,1042.0,1020.0,1023.0
2,BAJA CALIFORNIA SUR,111.0,133.0,123.0,117.0,138.0,138.0,119.0,147.0,139.0,124.0,130.0,149.0
3,CAMPECHE,231.0,196.0,249.0,244.0,231.0,195.0,263.0,219.0,198.0,219.0,216.0,239.0
4,COAHUILA,749.0,707.0,750.0,767.0,829.0,833.0,873.0,805.0,815.0,852.0,775.0,873.0
5,COLIMA,237.0,242.0,236.0,231.0,248.0,228.0,239.0,241.0,210.0,234.0,254.0,267.0
6,CHIAPAS,2585.0,2379.0,2750.0,2810.0,2679.0,2549.0,2691.0,2315.0,2323.0,2492.0,2311.0,2499.0
7,CHIHUAHUA,2464.0,2283.0,2558.0,2468.0,2710.0,2833.0,2649.0,2818.0,2653.0,2699.0,2579.0,2569.0
8,CIUDAD DE MEXICO,2278.0,2198.0,2398.0,2157.0,2319.0,2204.0,2183.0,2263.0,2354.0,2472.0,2375.0,2626.0
9,DURANGO,827.0,805.0,986.0,851.0,935.0,936.0,927.0,871.0,857.0,827.0,866.0,911.0


Finalmente si queremos obtener las 8 entidades con mas homicidios realizamos lo siguiente

In [67]:
dftarget=dftarget.sum(axis=1)
dftarget = dftarget.sort_values()# Ordenar 
dftarget = dftarget[::-1]# Invertir
dftarget = dftarget[:8]# Filtrar
dftarget

/tmp/ipykernel_4852/2530789100.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dftarget=dftarget.sum(axis=1)


14    84323.0
13    36296.0
15    32281.0
11    31717.0
7     31283.0
6     30383.0
8     27827.0
29    26764.0
dtype: float64

## Obtener el total de Homicidios cometidos por año.

In [68]:
dfyear = df_typeDelito1.groupby('AÑO').sum()
dfyear = dfyear.sum(axis=1)
dfyear

AÑO
1997    35341
1998    34444
1999    33242
2000    32018
2001    31185
2002    29144
2003    28330
2004    26530
2005    25771
2006    27552
2007    25133
2008    27759
2009    31546
2010    35056
2011    37408
2012    37849
2013    34576
2014    32399
dtype: int64

### Obtener la entidad con mayor Homicidios en cada año.

Para este caso usamos groupby con 2 columnas como parametros. Esto nos devuelve un df con 2 index para las filas. Posterior sumamos las columnas que son los meses.

Con la funcion .unstack cambiamos los indices para tener como index en las filas las entidades y como columnas tener los años.

Con la funcion .max() Buscamos el valor maximo de cada columna(año). 

Con la funcion .idmax() Buscamos los indices de fila del maximo encontrado en la columna.

Finalmente concatenamos para obtener nuestro resultado.


In [73]:
dfME = df_typeDelito1.groupby(['AÑO','Entidad']).sum()
dfME = dfME.sum(axis=1)
dfME = dfME.unstack(level=0)
m = dfME.max()
idm = dfME.idxmax()
maxEntidad=pd.concat([idm,m],axis=1)
maxEntidad

,0,1
AÑO,,
1997,MEXICO,6229
1998,MEXICO,6260
1999,MEXICO,6265
2000,MEXICO,6000
2001,MEXICO,6044
2002,MEXICO,6041
2003,MEXICO,6000
2004,MEXICO,5815
2005,MEXICO,5115
